In [1]:
import brightway2 as bw
bw.projects.set_current("exiobase_industry_workaround")
ex = bw.Database('EXIOBASE 2.2')
bi = bw.Database('biosphere3')
furniture_act = [act for act in ex if "Manufacture of furniture" in act['name'] and 'US' in act['location']][0]
method = ('ReCiPe Endpoint (H,A)', 'resources', 'total')

lca = bw.LCA({furniture_act:1}, method)
lca.lci()
lca.lcia()
lca.score

0.0

In [2]:
# Biosphere flows and their weights from ***recipe***
bw_method = bw.Method(method)
bio_from_method_codes = bw_method.load()
bio_from_method = {}
for el in bio_from_method_codes:
    bio_from_method.update({
        bw.get_activity(el[0])['name']: el[1]
    })
bio_from_method

{'Aluminium, 24% in bauxite, 11% in crude ore, in ground': 0.00418402983019,
 'Aluminium, in ground': 0.00418402983019,
 'Chromium, 25.5% in chromite, 11.6% in crude ore, in ground': 1.15756913164,
 'Coal, brown, in ground': 0.0283,
 'Coal, hard, unspecified, in ground': 0.054561,
 'Cobalt, in ground': 0.0469080296619,
 'Copper, 0.52% in sulfide, Cu 0.27% and Mo 8.2E-3% in crude ore, in ground': 1.98197244742,
 'Copper, 0.59% in sulfide, Cu 0.22% and Mo 8.2E-3% in crude ore, in ground': 1.98197244742,
 'Copper, 0.97% in sulfide, Cu 0.36% and Mo 4.1E-2% in crude ore, in ground': 1.98197244742,
 'Copper, 0.99% in sulfide, Cu 0.36% and Mo 8.2E-3% in crude ore, in ground': 1.98197244742,
 'Copper, 1.13% in sulfide, Cu 0.76% and Ni 0.76% in crude ore, in ground': 1.98197244742,
 'Copper, 1.18% in sulfide, Cu 0.39% and Mo 8.2E-3% in crude ore, in ground': 1.98197244742,
 'Copper, 1.42% in sulfide, Cu 0.81% and Mo 8.2E-3% in crude ore, in ground': 1.98197244742,
 'Copper, 2.19% in sulfide, Cu

In [3]:
# Exchanges from ***furniture demand***
furniture_exchanges = {}
for exc in list(furniture_act.exchanges()):
    furniture_exchanges.update({(exc.input['name']): exc.amount})
furniture_exchanges

{'Arsenic': 0.04011507117803164,
 'Benzo(a)pyrene': 0.016138121910241658,
 'Benzo(b)fluoranthene': 0.01360533055716102,
 'Benzo(k)fluoranthene': 0.01360533055716102,
 'Methane, fossil': 1088.59692032466,
 'Carbon monoxide, fossil': 996.9689798287598,
 'Carbon dioxide, fossil': 438.69694829978494,
 'Cadmium': 0.012023248379476171,
 'Chromium': 0.1325395019687227,
 'Copper': 0.0227247562137812,
 'Benzene, hexachloro-': 0.0,
 'Mercury': 0.004940682326362619,
 'Indeno(1,2,3-cd)pyrene': 0.004151980919714131,
 'Dinitrogen monoxide': 20.369656596897364,
 'Ammonia': 3.662065986972353,
 'NMVOC, non-methane volatile organic compounds, unspecified origin': 1371.4522059323353,
 'Nitrogen oxides': 33.72266102003201,
 'Nickel': 0.039861269279215655,
 'Nitrogen': 6.409545130830384,
 'PAH, polycyclic aromatic hydrocarbons': 0.5135407986628795,
 'Polychlorinated biphenyls': 0.00044823860673406533,
 'Dioxins, measured as 2,3,7,8-tetrachlorodibenzo-p-dioxin': 3.154873823813176e-08,
 'Particulates, > 2.5 

In [4]:
# Biosphere flows and their weights from ***furniture demand***
from pypardiso import spsolve
bio_weights_from_demand = lca.biosphere_matrix*spsolve(lca.technosphere_matrix, lca.demand_array)
bio_from_demand = {}
for i,bio_code in enumerate(lca.reverse_dict()[2].values()):
    bio_from_demand.update({bw.get_activity(bio_code)['name']: bio_weights_from_demand[i]})
bio_from_demand

{'Occupation, annual crop': 79875.79179382324,
 'Occupation, forest, unspecified': 3461843.0,
 'Occupation, pasture, man made': 324585.78125,
 'Ammonia': 24.751001834869385,
 'Arsenic': 0.9607412926852703,
 'Benzene, hexachloro-': 5.953756044618785e-06,
 'Benzo(a)pyrene': 0.19104035198688507,
 'Cadmium': 0.07206004578620195,
 'Carbon dioxide, fossil': 660588.2487182617,
 'Carbon monoxide, fossil': 5258.856689453125,
 'Chromium': 0.17444635927677155,
 'Copper': 1.0776505563408136,
 'Dinitrogen monoxide': 55.09036445617676,
 'Dioxins, measured as 2,3,7,8-tetrachlorodibenzo-p-dioxin': 3.8218705356030114e-07,
 'Lead': 0.5141026824712753,
 'Mercury': 0.04883421352133155,
 'Methane, fossil': 1125.4886093139648,
 'Nickel': 0.39935270696878433,
 'Nitrogen oxides': 1948.8334999084473,
 'NMVOC, non-methane volatile organic compounds, unspecified origin': 2138.36767578125,
 'PAH, polycyclic aromatic hydrocarbons': 0.5135408043861389,
 'Particulates, < 2.5 um': 233.72057723999023,
 'Particulates, 